Import Modules/Download Models

In [1]:
#install
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install scipy
#import
import glob
import pandas as pd
import re
import nltk
import sklearn
import scipy
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

#Download Models
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("stopwords")


You should consider upgrading via the 'C:\Users\小 周\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\小 周\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\小 周\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\小 周\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
C:\Users\小 周\AppData\Local\Temp\ipykernel_8208\19637222.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package wordnet to C:\Users\小
[nltk_data]     周\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\小
[nltk_data]     周\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\User

True

Read a txt file to form a text dataset

In [2]:
#Five types of press releases
categories = ["business", "entertainment", "politics", "sport", "tech"]
data_set = []
#Loop through the documents and add them to the array
for category in categories:

    for file_path in glob.glob(f"./data_set/bbc/{category}/*.txt"):
        with open(file_path, "r", encoding="ISO-8859-1") as file:
            content = file.read()
            data_set.append((content, category))
df_data_set = pd.DataFrame(data_set, columns=["content", "category"])

In [3]:
print(df_data_set.head())

                                             content  category
0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3  High fuel prices hit BA's profits\n\nBritish A...  business
4  Pernod takeover talk lifts Domecq\n\nShares in...  business


Cleansing of text data in datasets

In [4]:
#Text Cleaning Functions
def preprocess_text(text):
    #Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    #Convert to lowercase
    text = text.lower()
    #separate word
    tokens = word_tokenize(text)
    #Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    #morphological restoration
    lemmatizer = WordNetLemmatizer()
    lemmatizer_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    #Returns after joining words together
    return " ".join(lemmatizer_tokens)


df_data_set["processed_content"] = df_data_set["content"].apply(preprocess_text)


In [5]:
#View the cleaned text dataset
print(df_data_set.head())

                                             content  category  \
0  Ad sales boost Time Warner profit\n\nQuarterly...  business   
1  Dollar gains on Greenspan speech\n\nThe dollar...  business   
2  Yukos unit buyer faces loan claim\n\nThe owner...  business   
3  High fuel prices hit BA's profits\n\nBritish A...  business   
4  Pernod takeover talk lifts Domecq\n\nShares in...  business   

                                   processed_content  
0  ad sale boost time warner profit quarterly pro...  
1  dollar gain greenspan speech dollar hit highes...  
2  yukos unit buyer face loan claim owner embattl...  
3  high fuel price hit ba profit british airway b...  
4  pernod takeover talk lift domecq share uk drin...  


View the distribution of types

In [6]:
#Look at the number of press releases in each category to determine the method of dividing the data set
df_data_set["category"].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

Divide training set, development set, test set

In [7]:
#Divide the data set
#training set 60%
#Development set 20%
#Test set 20%
#Set a random seed to ensure that the dataset is the same each time it is divided up
train_dev_dataset, test_dataset = train_test_split(df_data_set, test_size=0.2, random_state=10)
train_dataset, dev_dataset = train_test_split(train_dev_dataset, test_size=0.25, random_state=10)
#Output the length of the three sets
print("train dataset:", len(train_dataset), "\ndevelopment dataset:", len(dev_dataset), "\ntest dataset",
      len(test_dataset))

train dataset: 1335 
development dataset: 445 
test dataset 445


Extraction characteristics

In [8]:
#word frequency
count_vectorizer = CountVectorizer()
#Learning and Transition
train_count_vectorizer = count_vectorizer.fit_transform(train_dataset["processed_content"])
dev_count_vectorizer = count_vectorizer.transform(dev_dataset["processed_content"])
test_count_vectorizer = count_vectorizer.transform(test_dataset["processed_content"])

In [9]:
#binary group frequency
N_gram_vectorizer = CountVectorizer(ngram_range=(2, 2))
#Learning and Transition
train_N_gram_vectorizer = N_gram_vectorizer.fit_transform(train_dataset["processed_content"])
dev_N_gram_vectorizer = N_gram_vectorizer.transform(dev_dataset["processed_content"])
test_N_gram_vectorizer = N_gram_vectorizer.transform(test_dataset["processed_content"])

In [10]:
#TF-IDF features
tf_idf_vectorizer = TfidfVectorizer(max_features=10000)
#Learning and Transition
train_tf_idf_vectorizer = tf_idf_vectorizer.fit_transform(train_dataset["processed_content"])
dev_tf_idf_vectorizer = tf_idf_vectorizer.transform(dev_dataset["processed_content"])
test_tf_idf_vectorizer = tf_idf_vectorizer.transform(test_dataset["processed_content"])

In [11]:
#Categorised Tags
train_category = train_dataset["category"]
dev_category = dev_dataset["category"]
test_category = test_dataset["category"]

In [12]:
#Print the dimensions of the data
print("word-count:",train_count_vectorizer.shape)
print("N-gram:",train_N_gram_vectorizer.shape)
print("tf-idf:",train_tf_idf_vectorizer.shape)

word-count: (1335, 22370)
N-gram: (1335, 211851)
tf-idf: (1335, 10000)


Perform feature combinations

In [13]:
#train
train_combined_cnt = hstack([train_count_vectorizer, train_N_gram_vectorizer, train_tf_idf_vectorizer])
train_combined_cn = hstack([train_count_vectorizer, train_N_gram_vectorizer])
train_combined_ct = hstack([train_count_vectorizer, train_tf_idf_vectorizer])
train_combined_nt = hstack([train_N_gram_vectorizer, train_tf_idf_vectorizer])
#develop
dev_combined_cnt = hstack([dev_count_vectorizer, dev_N_gram_vectorizer, dev_tf_idf_vectorizer])
dev_combined_cn = hstack([dev_count_vectorizer, dev_N_gram_vectorizer])
dev_combined_ct = hstack([dev_count_vectorizer, dev_tf_idf_vectorizer])
dev_combined_nt = hstack([dev_N_gram_vectorizer, dev_tf_idf_vectorizer])
#test
test_combined_cnt = hstack([test_count_vectorizer, test_N_gram_vectorizer, test_tf_idf_vectorizer])
test_combined_cn = hstack([test_count_vectorizer, test_N_gram_vectorizer])
test_combined_ct = hstack([test_count_vectorizer, test_tf_idf_vectorizer])
test_combined_nt = hstack([test_N_gram_vectorizer, test_tf_idf_vectorizer])

Accuracy, macro-averaged precision, macro-recall, macro-averaged f1-score function

In [14]:
#Function feedback (label, predicted label, learning model used)
def feedback(category, preictions, x):
    #Accuracy
    accuracy = accuracy_score(category, preictions)
    #macro-averaged precision
    macro_precision = precision_score(category, preictions, average="macro")
    #macro-recall
    macro_recall = recall_score(category, preictions, average="macro")
    #macro-averaged f1-score
    macro_f1 = f1_score(category, preictions, average="macro")
    #output test results
    print(x)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"macro-averaged precision:{macro_precision * 100:.2f}%")
    print(f"macro-recall:{macro_recall * 100:.2f}%")
    print(f"macro-averaged f1-score:{macro_f1 * 100:.2f}%")

Model training function

In [15]:
nb_classifier = MultinomialNB()


#MultinomialNB model
#nb_classifier_test(Training set features, training set labels, features to be predicted, labels to be tested)
def nb_classifier_test(train_combined, train_category, dev_combined, dev_category):
    #Learning using features and labels
    nb_classifier.fit(train_combined, train_category)
    #Prediction using learnt models
    nb_preictions = nb_classifier.predict(dev_combined)
    feedback(dev_category, nb_preictions, "NB:")

In [16]:
decision_tree_classifier = DecisionTreeClassifier(random_state=11)


#Decision Trees model
#decision_tree_classifier_test(Training set features, training set labels, features to be predicted, labels to be tested)
def decision_tree_classifier_test(train_combined, train_category, dev_combined, dev_category):
    #Learning using features and labels
    decision_tree_classifier.fit(train_combined, train_category)
    #Prediction using learnt models
    decision_tree_preictions = decision_tree_classifier.predict(dev_combined)
    feedback(dev_category, decision_tree_preictions, "Decision-Tree:")

In [17]:
random_forest_classifier = RandomForestClassifier(random_state=11)


#Random Forest model
#random_forest_classifier_test(Training set features, training set labels, features to be predicted, labels to be tested)
def random_forest_classifier_test(train_combined, train_category, dev_combined, dev_category):
    #Learning using features and labels
    random_forest_classifier.fit(train_combined, train_category)
    #Prediction using learnt models
    random_forest_preictions = random_forest_classifier.predict(dev_combined)
    feedback(dev_category, random_forest_preictions, "Random-Forest:")

Testing with models

In [18]:
#Detecting Models Using train Sets
#MultinomialNB
start_time = time.time()
nb_classifier_test(train_count_vectorizer, train_category, train_count_vectorizer, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#Decision Trees
start_time = time.time()
decision_tree_classifier_test(train_count_vectorizer, train_category, train_count_vectorizer, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#Random Forest
start_time = time.time()
random_forest_classifier_test(train_count_vectorizer, train_category, train_count_vectorizer, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")

NB:
Accuracy: 99.63%
macro-averaged precision:99.61%
macro-recall:99.62%
macro-averaged f1-score:99.61%
Using time:0.02 s

Decision-Tree:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:0.27 s

Random-Forest:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:1.05 s


Feature filtering

In [19]:
#N_gram_vectorizer
start_time = time.time()
print("N_gram_vectorizer:")
nb_classifier_test(train_N_gram_vectorizer, train_category, train_N_gram_vectorizer, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#tf_idf_vectorizer
start_time = time.time()
print("tf_idf_vectorizer:")
nb_classifier_test(train_tf_idf_vectorizer, train_category, train_tf_idf_vectorizer, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#count_vectorizer + N_gram_vectorizer + tf_idf_vectorizer
start_time = time.time()
print("combined_cnt:")
nb_classifier_test(train_combined_cnt, train_category, train_combined_cnt, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#count_vectorizer + N_gram_vectorizer
start_time = time.time()
print("combined_cn:")
nb_classifier_test(train_combined_cn, train_category, train_combined_cn, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#count_vectorizer + tf_idf_vectorizer
start_time = time.time()
print("combined_ct:")
nb_classifier_test(train_combined_ct, train_category, train_combined_ct, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")
print()
#N_gram_vectorizer + tf_idf_vectorizer
start_time = time.time()
print("combined_nt:")
nb_classifier_test(train_combined_nt, train_category, train_combined_nt, train_category)
end_time = time.time()
print(f"Using time:{end_time - start_time:.2f} s")

N_gram_vectorizer:
NB:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:0.05 s

tf_idf_vectorizer:
NB:
Accuracy: 99.03%
macro-averaged precision:99.03%
macro-recall:98.96%
macro-averaged f1-score:98.99%
Using time:0.02 s

combined_cnt:
NB:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:0.05 s

combined_cn:
NB:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:0.05 s

combined_ct:
NB:
Accuracy: 99.70%
macro-averaged precision:99.67%
macro-recall:99.70%
macro-averaged f1-score:99.69%
Using time:0.03 s

combined_nt:
NB:
Accuracy: 100.00%
macro-averaged precision:100.00%
macro-recall:100.00%
macro-averaged f1-score:100.00%
Using time:0.05 s


Testing models for overfitting using cross-validation and development sets

In [20]:
#train set cross-validation
#count_vectorizer
scores = cross_val_score(nb_classifier, train_count_vectorizer, train_category, cv=10, scoring="accuracy")
print("count_vectorizer:")
print("Average accuracy:%0.2f" % (scores.mean() * 100), "%")
print("Standard deviation of accuracy: %0.2f" % (scores.std() * 100), "%")
print()
#tf_idf_vectorizer
scores = cross_val_score(nb_classifier, train_tf_idf_vectorizer, train_category, cv=10, scoring="accuracy")
print("tf_idf_vectorizer:")
print("Average accuracy:%0.2f" % (scores.mean() * 100), "%")
print("Standard deviation of accuracy: %0.2f" % (scores.std() * 100), "%")
print()
#count_vectorizer + tf_idf_vectorizer
scores = cross_val_score(nb_classifier, train_combined_ct, train_category, cv=10, scoring="accuracy")
print("count_vectorizer + tf_idf_vectorizer:")
print("Average accuracy:%0.2f" % (scores.mean() * 100), "%")
print("Standard deviation of accuracy: %0.2f" % (scores.std() * 100), "%")

count_vectorizer:
Average accuracy:97.15 %
Standard deviation of accuracy: 1.00 %

tf_idf_vectorizer:
Average accuracy:96.55 %
Standard deviation of accuracy: 1.68 %

count_vectorizer + tf_idf_vectorizer:
Average accuracy:97.23 %
Standard deviation of accuracy: 1.46 %


In [21]:
#Testing with the development set
nb_classifier_test(train_count_vectorizer, train_category, dev_count_vectorizer, dev_category)

NB:
Accuracy: 97.08%
macro-averaged precision:97.14%
macro-recall:96.98%
macro-averaged f1-score:97.05%


Prediction of test sets using models

In [22]:
#Test Set Validation
nb_classifier_test(train_count_vectorizer, train_category, test_count_vectorizer, test_category)

NB:
Accuracy: 98.43%
macro-averaged precision:98.36%
macro-recall:98.51%
macro-averaged f1-score:98.42%
